In [4]:
import string
import json
import time
import requests
from calendar import monthrange
from selenium import webdriver
from bs4 import BeautifulSoup

#today = date.today()
SAVE_FOLDER = r'C:\tmp'
BASE_URI = 'https://bvger.weblaw.ch/'

DATE_suffix1 = 'dashboard?guiLanguage=de&filters=%5B%5B%22rulingDate%22%2C%5B%7B%22from%22%3A%22'
DATE_suffix2 = 'T00%3A00%3A00.000Z%22%2C%22to%22%3A%22'
DATE_suffix3 = 'T22%3A59%3A59.999Z%22%7D%5D%5D%5D&sort-field=relevance&sort-direction=relevance'

In [5]:
def getdays(year):
    date_list = []
    for i in range(12):
        m_fillerzero = ''
        if i <= 8:
            m_fillerzero = '0'
        for j in range(monthrange(year, (i+1))[1]):
            fillerzero = ''
            if j <= 8:
                fillerzero = '0'
            date_list.append(str(year)+'-'+m_fillerzero+str(i+1)+'-'+fillerzero+str(j+1))
    return date_list

def getfilters(start, end):
    date_filters = []
    for i in range(start,end):
        date_filters += getdays(i)
    return date_filters
    
myfilters = getfilters(2007,2024)

filtered_urls = []
for i in range(len(myfilters)):
    if (i+2) > len(myfilters):
        break
    filtered_urls.append(BASE_URI+DATE_suffix1+myfilters[i]+DATE_suffix2+myfilters[i+1]+DATE_suffix3)

In [ ]:
case_urls = []

def scrap_link(html_source):
    url_list = []
    soup = BeautifulSoup(html_source, 'html.parser').body
    anchor_elements = soup.select('a[href]')
    if len(anchor_elements) >= 1:
        for element in anchor_elements:
            url = str(element).split(' ')[1].split('"')[1]
            if url[1] == 'c':
                url_list.append((BASE_URI[:-1]+url))

        return url_list
    del soup, anchor_elements, url_list
    
for fu in filtered_urls:
    options = webdriver.EdgeOptions()
    driver = webdriver.Edge(options=options)
    #driver.manage().window().minimize()
    driver.get(fu)
    scroll_pause_time = 1  # Pause between each scroll
    screen_height = driver.execute_script("return window.screen.height;")  # Browser window height
    i = 1
    while True:
        # Scroll down
        driver.execute_script(f"window.scrollTo(0, {screen_height * i});")
        i += 1
        time.sleep(scroll_pause_time)

        # Check if reaching the end of the page
        scroll_height = driver.execute_script("return document.body.scrollHeight;")
        if screen_height * i > scroll_height:
            break

    html = driver.page_source
    driver.quit()
    new_urls = scrap_link(html)
    if new_urls != None:
         case_urls += new_urls
    #del driver

In [ ]:
abschnitt_namen = ['Sachverhalt','Erwaegunen','Urteil']
sv = {
  'de': 'Sachverhalt:',
  'fr': 'Faits:',
  'it': 'Ritenuto in fatto:'
}
ew = {
  'de': 'Erwägung:',
  'fr': 'Droit:',
  'it': 'Considerando in diritto:'
}
vd = {
  'de': 'erkennt das Bundesverwaltungsgericht:',
  'fr': 'fédéral prononce:',
  'it': 'federale pronuncia:'        
}
crit = [sv,ew,vd]

In [46]:
def part1_dissection(datablock):
    alf_rev = string.ascii_uppercase[::-1]
    letters = []
    for letter in alf_rev:
        if (datablock.find((letter+'.')) != -1):
            letters.append(letter)
    letters = letters[::-1]
    
    actual_letters = []
    for i in range(len(letters)):
        if letters[i] == string.ascii_uppercase[i]:
            actual_letters.append(letters[i])

    prv_idx = -1
    cur_idx = 0
    paragraphs_part1 = []
    for l in actual_letters[::-1]:
        if datablock.find((l+'. ')) != -1:
            cur_idx = datablock.index((l+'. '))
            paragraphs_part1.append(datablock[cur_idx:prv_idx])
            prv_idx = cur_idx
        else:
            for abc in string.ascii_lowercase[::-1]:
                if datablock.find(l+'.'+abc) != -1:
                    cur_idx = datablock.index(l+'.'+abc)
                    paragraphs_part1.append(datablock[cur_idx:prv_idx])
                    prv_idx = cur_idx
            paragraphs_part1.append(l+'.')

    return paragraphs_part1[::-1]

In [ ]:
def part2_dissection(datablock):
    i = 1
    found_is = []
    while (datablock.find(str(i)+'. ') != -1):
        found_is.append(i)
        i += 1

    index_list = []
    for par_n in found_is:
        cur_idx = datablock.index(str(par_n)+'. ')
        if len(index_list) >= 1:
            cur_idx = datablock[index_list[-1]:-1].index(str(par_n)+'. ') + index_list[-1]
            if datablock[cur_idx-1] == '.':
                cur_idx = datablock[cur_idx+1:-1].index(str(par_n)+'. ') + (cur_idx+1)
        index_list.append(cur_idx)
    index_list.append(-1)

    paragraphs_part2 = []
    for i in range(len(index_list)):
        if i+1 < len(index_list):
            paragraphs_part2.append(datablock[index_list[i]:index_list[i+1]])

    sub_is =[]
    sub_is_main = []
    for par in part2_paragraphs:
        i = 1
        dot_idx = par.find('.')+1
        pre_i = par[0:dot_idx]
        while (par.find(pre_i+str(i)) != -1):
            if pre_i[:-1] not in sub_is:
                sub_is.append(pre_i)#[:-1])
            sub_is.append(pre_i+str(i))
            if int(pre_i[:-1])-1 not in sub_is_main:
                sub_is_main.append(int(pre_i[:-1])-1)
            i += 1
    print(sub_is)
    print(sub_is_main)

    sub_pars =[]
    prev_par_n = -1
    for i in range(len(sub_is_main)-1,-1,-1):
        sub_par_parts = []
        pre_idx = -1
        for j in range(len(sub_is)-1,-1,-1):
            pre_i = sub_is[j][0:sub_is[j].find('.')]
            if paragraphs_part2[sub_is_main[i]].find(sub_is[j]+'. ') != -1 and int(pre_i) == (sub_is_main[i]+1):
                idx = paragraphs_part2[sub_is_main[i]].find(sub_is[j]+'. ')
                sub_par_parts.append(paragraphs_part2[sub_is_main[i]][idx:pre_idx])
                pre_idx = idx
        sub_pars.append(sub_par_parts[::-1])
    sub_pars = sub_pars[::-1]

    for i in range(len(sub_is_main)):
        paragraphs_part2[sub_is_main[i]] = sub_pars[i]
    return paragraphs_part2

In [ ]:
def part3_dissection(datablock):
    i = 1
    found_is = []
    while (datablock.find(str(i)+'. ') != -1):
        found_is.append(i)
        i += 1

    index_list = []
    for par_n in found_is:
        cur_idx = datablock.index(str(par_n)+'. ')
        if len(index_list) >= 1:
            cur_idx = datablock[index_list[-1]:-1].index(str(par_n)+'. ') + index_list[-1]
        index_list.append(cur_idx)
    index_list.append(-1)

    part3_paragraphs = []
    for i in range(len(index_list)):
        if i+1 < len(index_list):
            part3_paragraphs.append(datablock[index_list[i]:index_list[i+1]])

    return part3_paragraphs

In [ ]:
def json_data_prep(data):
    #json_data = {}
    j_data ={}
    for d in data:
        if type(d) == list:
            sub_json ={}
            obj = ''
            for sub_d in d:
                #if type(li_li) == list:                
                #    for li_li_li in li_li:
                #        par2_json[li_li[0:li_li.find('. ')+1]] = li_li[li_li.find('. ')+2:-1]
                #else:
                sub_json[sub_d[0:sub_d.find('. ')+1]] = sub_d[sub_d.find('. ')+2:-1]
                if obj == '':
                    obj = sub_d[0:sub_d.find('. ')-1]
            j_data[obj] = [sub_json]
        else:
            j_data[d[0:d.find('. ')+1]] = d[d.find('. ')+2:-1]
    #json_data[partname] = [j_data]
    return j_data #json_data

def json_maker (data,doctitle,savefolder):      
    json_fn = savefolder+'\\'+doctitle+'.json'
    with open(json_fn, 'w') as f:
        json.dump(data, f)

In [3]:
def la_cuisine(bouillon,lang,doctitle):
    criteria1 = crit[0][lang]
    criteria2 = crit[1][lang]
    criteria3 = crit[2][lang]

    str_thepart  = str(bouillon)
    cr3_idx = str_thepart.index(criteria3)
    cr2_idx = str_thepart.index(criteria2)
    cr1_idx = str_thepart.index(criteria1)
    part3 = (str_thepart[cr3_idx:]).replace('</div>','').replace('</p>','').replace('<p>',' ').replace('  ',' ')
    part2_idx = str_thepart[cr2_idx:cr3_idx][::-1].index('.')-1
    part2 = (str_thepart[cr2_idx:cr3_idx-part2_idx]).replace('</p>','').replace('<p>',' ').replace('  ',' ')
    part1_idx = str_thepart[cr1_idx:cr2_idx][::-1].index('.')-1
    part1 = (str_thepart[cr1_idx:cr2_idx-part1_idx]).replace('</p>','').replace('<p>',' ').replace('  ',' ')
    part3 = ' '.join(part3.split())
    part2 = ' '.join(part2.split())
    part1 = ' '.join(part1.split())
    
    part1_new = part1_dissection(part1)
    part2_new = part2_dissection(part2)
    part3_new = part3_dissection(part3)
    
    final_data = jason_data_prep(part1_new,part2_new,part3_new)
    json_data['name'] = doctitle
    json_data[abschnitt_namen[0]]= json_data_prep(part1_new,abschnitt_namen[0])
    json_data[abschnitt_namen[1]]= json_data_prep(part2_new,abschnitt_namen[1])
    json_data[abschnitt_namen[2]]= json_data_prep(part3_new,abschnitt_namen[2])
    
    jason_maker(json_data,doctitle,SAVE_FOLDER)
    return True

'\nIT \nRitenuto in fatto:\nConsiderando in diritto:\nPer questi motivi, il Tribunale amministrativo federale pronuncia:\n\nFR\nFaits:\nDroit:\nPar ces motifs, le Tribunal administratif fédéral prononce :\n\nDE\nSachverhalt:\nDas Bundesverwaltungsgericht zieht in Erwägung:\nDemnach erkennt das Bundesverwaltungsgericht:\n'

In [ ]:
for case_url in case_urls:
    driver = webdriver.Edge()
    driver.get(case_url)
    time.sleep(1)
    html = driver.page_source
    driver.quit()
    soup = BeautifulSoup(html, 'html.parser')
    ent_name = soup.title.string
    thepart = (soup.find("div",{"id":"customContentSegment"})).find_all('div')[-2]
    lng = soup.find('a',{"class":"navig loose selected_page"}).string
    la_cuisine(thepart,lng,ent_name)